In [7]:
import pandas as pd
import numpy as np
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
init_notebook_mode()
import plotly.express as px
pd.options.plotting.backend = "plotly"
import io
import plotly.io as pio
import re

In [8]:
sims_path = '../../../simulaciones/generador_pulsos_v2_wrk/'
meas_path = '../../../mediciones/2022_12_27_INTI'

vcc_5_duty_50 = {
    'simulations_path': f'{sims_path}/simulations_vcc_5_duty_50.txt',
    'measurements_path': f'{meas_path}/5V/csv/duty_50.csv',
    'title': 'Vcc: 5V, duty: 50%'
}

vcc_5_duty_70 = {
    'simulations_path': f'{sims_path}/simulations_vcc_5_duty_70.txt',
    'measurements_path': f'{meas_path}/5V/csv/duty_70.csv',
    'title': 'Vcc: 5V, duty: 70%'
}

vcc_7_duty_70 = {
    'simulations_path': f'{sims_path}/simulations_vcc_7_duty_70.txt',
    'measurements_path': f'{meas_path}/7V/csv/duty_70.csv',
    'title': 'Vcc: 7V, duty: 70%'
}

# plots = vcc_5_duty_50
plots = vcc_5_duty_70
#plots = vcc_7_duty_70

In [9]:
simulations_path = plots['simulations_path']
measurements_path = plots['measurements_path']
title = plots['title']

lets start by reading measurements

In [10]:
data_measurements = pd.read_csv(
    filepath_or_buffer = measurements_path, 
    sep = ',', 
    dtype = np.float64,
    header = None,
    usecols = [3, 4],
    names = ['time', 'v']
).set_index('time')
display(data_measurements)

,v
time,
-1.200000e-09,0.001153
-1.198000e-09,0.000970
-1.196000e-09,0.000802
-1.194000e-09,0.000647
-1.192000e-09,0.000506
...,...
1.290000e-09,-0.006581
1.292000e-09,-0.006806
1.294000e-09,-0.007003


In [11]:
# read the contents of the file
with open(simulations_path, 'r') as file:
    contents = file.read()

# split the contents by empty lines
blocks = contents.split('\n\n')[:-1]

# create a dataframe from each block
df_list = []
for block in blocks:
    # skip any empty rows
    rows = [row for row in block.split('\n')]
    # create a dataframe from the rows
    df = pd.read_csv(
        filepath_or_buffer = io.StringIO('\n'.join(rows)), 
        sep='\t',
        dtype = np.float64,
    ).set_index('time')
    df_list.append(df)

# print the resulting dataframes
df_sch_ideal, df_layout = df_list

print(df_sch_ideal)
print(df_layout)

              Vo_sch_ideal
time                      
0.000000e+00 -1.712858e-24
1.000000e-13  1.026451e-08
2.000000e-13  1.022454e-08
5.000000e-13  9.907097e-09
1.400000e-12  9.238367e-09
...                    ...
9.996618e-08 -4.422457e-06
9.997618e-08  4.422354e-06
9.998618e-08 -4.422252e-06
9.999618e-08  4.422157e-06
1.000000e-07 -4.421811e-06

[10050 rows x 1 columns]
                 Vo_layout
time                      
0.000000e+00 -3.434159e-20
1.000000e-13  1.969346e-07
2.000000e-13  3.897359e-07
5.000000e-13  9.709096e-07
1.400000e-12  2.740389e-06
...                    ...
9.996618e-08 -6.656036e-06
9.997618e-08 -7.628188e-06
9.998618e-08 -6.361823e-06
9.999618e-08 -5.478932e-06
1.000000e-07 -4.913855e-06

[10050 rows x 1 columns]


In [12]:
data_to_plot = (
    {
        'df': df_sch_ideal,
        'name': 'Esquemático ideal',
        'sample_time': 10e-12,
    },
    {
        'df': df_layout,
        'name': 'Layout',
        'sample_time': 10e-12,
    },
    {
        'df': data_measurements,
        'name': 'Medición',
        'sample_time': 2e-12,
    },
)

Find the time length of the captured data

In [13]:
get_pkpk = lambda x: np.max(x)-np.min(x)
measurements_length = get_pkpk(data_measurements.index)

In [14]:
display(measurements_length)

2.4980000000000003e-09

lets plot measurements alongside simulation

In [15]:
filenamesize = lambda s: re.sub("_+", "_", s.replace(" ", "_").replace(",", "_").replace("%", "_").replace(":", "_"))

In [16]:
def extract_centered(df, T):
    # get the name of the single column in the DataFrame
    column_name = df.columns[0]

    # find the index of the maximum value in the column
    max_value_idx = df[column_name].idxmax()

    # calculate the start and end times for the subset of data
    start_time = max_value_idx - T/2
    end_time = max_value_idx + T/2

    # extract the subset of data
    subset_df = df.loc[start_time:end_time]
    
    # scale the index so it starts at 0
    subset_df.index = subset_df.index - start_time

    return subset_df

In [17]:
fig = go.Figure()

# lambda for plotting centered at max with T length
plot_centered = lambda df, T, name: go.Scatter(
    x = extract_centered(df, T).index, 
    y = extract_centered(df, T)[df.columns[0]], 
    name = name,
)

# plot all data
for data in data_to_plot:
    fig.add_trace(plot_centered(
        data['df'],
        measurements_length, 
        data['name'],
    ))

# set title and labels
title_centered = lambda s: {'text': s, 'font': {'size': 20}, 'x':0.5, 'y':0.95, 'xanchor': 'center', 'yanchor': 'top'}
fig.update_layout(
    xaxis_title = 'Tiempo [s]', 
    yaxis_title = 'Tensión [V]',
    title = title_centered(f'{title}'),
)

# show plot
fig.show()

# save the plot
pio.write_image(fig, f'results/{filenamesize(title)}time_domain.png')

Lets start by defining some utility functions

In [18]:
def get_spectrum(x, sample_time):
    w = np.hanning(len(x.shape))
    x_w = np.multiply(x,w)
    X_W = np.fft.rfft(x_w)
    X_W_dB = 10*np.log10(np.abs(X_W))
    freq = np.fft.rfftfreq(x_w.shape[-1])/sample_time
    return pd.DataFrame({'frequency': freq, 'magnitude': X_W_dB})

In [19]:
def get_bandwidth(df, threshold_dB):
    """
    Find the frequency at which the magnitude drops by a specified threshold (in dB) with respect to magnitude at frequency 0.
    Assumes that 'df' is a pandas dataframe with columns 'frequency' and 'magnitude' (in dB).
    Returns the frequency value as a float.
    """
    # find the row with frequency equal to 0 and get the magnitude value
    mag_at_0 = df.loc[df['frequency'] == 0, 'magnitude'].iloc[0]

    # calculate the threshold magnitude value (specified threshold less than the magnitude at frequency 0)
    thresh_mag = mag_at_0 - threshold_dB

    # find the row with the first occurrence of a magnitude value below the threshold, and get the corresponding frequency value
    first_below_thresh = df.loc[df['magnitude'] < thresh_mag, 'frequency'].iloc[0]

    return float(first_below_thresh)

In [20]:
def get_dataframe_spectrum(df, sample_time):
    return get_spectrum(df[df.columns[0]], sample_time)

In [21]:
fig = go.Figure()

for data in data_to_plot:
    
    # get the spectrum
    spectrum = get_dataframe_spectrum(
        df = data['df'],
        sample_time = data['sample_time']
    )
    
    # get the 3 dB BW in GHz
    BW_3_dB = get_bandwidth(spectrum, 3)*1e-9    
    
    # plot the spectrum
    fig.add_trace(go.Scatter(
        x = spectrum.frequency*1e-9, 
        y = spectrum.magnitude, 
        mode = 'lines',
        name = data['name'],
    ))

    # plot a line at 3 dB BW
    fig.add_trace(go.Scatter(
        x = [BW_3_dB, BW_3_dB], 
        y = [min(fig.data[0].y), max(fig.data[0].y)],
        mode = 'lines', 
        line = dict(dash='dash'),
        name = f'{data["name"]} 3 dB BW = {BW_3_dB:.2} GHz'
    ))

fig.update_layout(
    xaxis_title = "Freq [GHz]",
    yaxis_title = "Magnitude [dB]",
    xaxis_range = [0, 20],
    title = title_centered(f'PSD, {title}'),
)

fig.show()

# save the plot
pio.write_image(fig, f'results/{filenamesize(title)}psd.png')

In [22]:
fig = go.Figure()

data = data_to_plot[2]
    
# get the spectrum
spectrum = get_dataframe_spectrum(
    df = data['df'],
    sample_time = data['sample_time']
)

# get the 3 dB BW in GHz
BW_3_dB = get_bandwidth(spectrum, 3)*1e-9    

# plot the spectrum
fig.add_trace(go.Scatter(
    x = spectrum.frequency*1e-9, 
    y = spectrum.magnitude, 
    mode = 'lines',
    name = data['name'],
))

# plot a line at 3 dB BW
fig.add_trace(go.Scatter(
    x = [BW_3_dB, BW_3_dB], 
    y = [min(fig.data[0].y), max(fig.data[0].y)],
    mode = 'lines', 
    line = dict(dash='dash'),
    name = f'{data["name"]} 3 dB BW = {BW_3_dB:.2} GHz'
))

fig.update_layout(
    xaxis_title = "Freq [GHz]",
    yaxis_title = "Magnitude [dB]",
    xaxis_range = [0, 20],
    title = title_centered(f'PSD, {title}'),
)

fig.show()

In [23]:
def find_threshold_index(df, threshold):
    # Normalize column
    normalized_col = df.iloc[:,0] / df.iloc[:,0].sum()
    # Compute cumulative sum of normalized column
    cumsum = np.cumsum(normalized_col)
    # Find index where cumsum exceeds threshold
    idx = np.argmax(cumsum > threshold)
    return idx

In [24]:
spectrum = get_dataframe_spectrum(
        df = data_to_plot[2]['df'],
        sample_time = data['sample_time']
    )

spectrum_linear = spectrum
# square to get the power
spectrum_linear['magnitude'] = (10**(spectrum_linear['magnitude']/10))**2
spectrum_linear = spectrum_linear.set_index('frequency')

find_threshold_index(spectrum_linear, 0.9999)

11

In [25]:
spectrum_linear.plot()